# Localización acústica

En esta practica vamos a mostrar el flujo de trabajo para hacer localizacion de una fuente de sonido a partir de ARUs sincronizadas, para ello es necesario:

1) Despliegue de dispositivos de grabación sincronizables en el tiempo en ubicaciones estáticas y conocidas.

2) Sincronizar las grabaciones.

3) Etiquetado de las detecciones del sonido de interés en el audio.

4) Estimación de los delays temporales de llegada (TDOA) del sonido de interés entre varios micrófonos.

5) Localización de la fuente sonora a partir de los TDOA.

6) Evaluar la fiabilidad de estas localizaciones.


In [ ]:
import opensoundscape
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import subprocess
from opensoundscape.localization import SynchronizedRecorderArray
import pytz
from datetime import datetime, timedelta

%config InlineBackend.figure_format = 'retina'

## Descarga de datos

In [ ]:
folder="/Users/santiagoruiz/Documents/AI_Andes/Sesiones_practicas/Tercera_practica/Datos"

# Usa tu carpeta home o cualquier otra con permisos de escritura
output_dir = Path.home() / folder 
tar_path = output_dir / "localization_files.tar"

# Asegúrate de que el directorio existe
output_dir.mkdir(parents=True, exist_ok=True)

# Descargar
subprocess.run(
    ["curl", "-L", 
     "https://drive.google.com/uc?export=download&id=1M4yKM8obqiY0FU2qEriINBDSWtQGqN8E",
     "-o", str(tar_path)]
)

# Descomprimir
subprocess.run(["tar", "-xzf", str(tar_path)], cwd=str(output_dir))

# Borrar el archivo .tar
tar_path.unlink()


## Preparar coordenadas

In [ ]:
# El primer tipo de archivo que necesitamos es una tabla con las coordenadas de 1 archivo de sonido por ARU. 
aru_coords = pd.read_csv(folder + "/aru_coords.csv", index_col=0) 
aru_coords

In [ ]:
array = SynchronizedRecorderArray(aru_coords)

# Carga tabla con las detecciones (Obtenidas por cualquier tipo de detector)
detections = pd.read_csv(folder + "/detections.csv")

# Añade informacion de fecha, hora y zona horaria si el audio no lo contiene como metadatos
local_timestamp = datetime(2022, 2, 7, 20, 0, 0) 
local_timezone = pytz.timezone("US/Eastern")
detections["start_timestamp"] = [
    local_timezone.localize(local_timestamp) + timedelta(seconds=s)
    for s in detections["start_time"]
]

#detections['file'] = folder + "/"+ detections['file']
# Modifica el index de la tabla como multi-index 
detections = detections.set_index(["file", "start_time", "end_time", "start_timestamp"])

detections

## Localizacion de detecciones

Antes de continuar es necesario definir 2 parametros:


1) **min_n_receivers:** Número mínimo de receptores en los que debe detectarse un sonido para que se intente la localización. Debe ser al menos n+2 para localizar un punto en n dimensiones. Si tiene una cuadrícula de localización densa y espera que su sonido se escuche en más receptores, puede aumentar este número y puede mejorar la precisión de las estimaciones de localización.

2) **max_receiver_dist+** Los TDOAs se estiman entre pares de receptores. Sólo se utilizarán para la estimación TDOAs los receptores que se encuentren a una distancia máxima entre sí. Si max_receiver_dist=100, entonces si 2 receptores >100 de distancia ambos contienen detecciones del mismo sonido, no se estimará un TDOA entre ellos. Esto es útil para separar múltiples sonidos simultáneos en diferentes ubicaciones si está desplegando un gran array.

In [ ]:
min_n_receivers = 4  
max_receiver_dist = 100 
position_estimates = array.localize_detections(
    detections, min_n_receivers=min_n_receivers, max_receiver_dist=max_receiver_dist
)

El output final es una lista de objetos "PositionEstimate" que contienen los Sound events, veamos un ejemplo

In [ ]:
example = position_estimates[0]
print(f"El tiempo de inicio de la detección es: {example.start_timestamp}")
print(f"Esta es una deteccion de la clase/especie: {example.class_name}")
print(
    f"La duración de la ventana temporal en la que se detectó el sonido: {example.duration}"
)
print(f"La ubicacion estimada del sonido: {example.location_estimate}")
print(f"La lista de ARUs en la que el sonido fue detectado: \n{example.receiver_files}")
print(f"Los TDOAs estimados: \n{example.tdoas}")
print(f"Los valores normalizados de cross-correlation: \n{example.cc_maxs}")

In [ ]:
plt.plot(aru_coords["x"], aru_coords["y"], "^", label="ARU")
plt.scatter(
    x=example.location_estimate[0],
    y=example.location_estimate[1],
    color="red",
    label=f"{example.class_name}",
)
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left")
plt.show()